# Quantum Espresso Energy Volume Curve Workflow with pyiron_base

## Define workflow with pyiron_base

In [1]:
import numpy as np
from pyiron_base import Project, job
from python_workflow_definition.pyiron_base import write_workflow_json

In [2]:
from quantum_espresso_workflow import (
    calculate_qe as _calculate_qe, 
    generate_structures as _generate_structures, 
    get_bulk_structure as _get_bulk_structure, 
    plot_energy_volume_curve as _plot_energy_volume_curve,
)

In [3]:
workflow_json_filename = "pyiron_base_qe.json"

In [4]:
calculate_qe = job(_calculate_qe, output_key_lst=["energy", "volume", "structure"])
generate_structures = job(_generate_structures)
plot_energy_volume_curve = job(_plot_energy_volume_curve)
get_bulk_structure = job(_get_bulk_structure)

In [5]:
pseudopotentials = {"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}

In [6]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

In [7]:
structure = get_bulk_structure(
    element="Al",
    a=4.05,
    cubic=True,
    pyiron_project=pr,
)

In [8]:
calc_mini = calculate_qe(
    working_directory="mini",
    input_dict={
        "structure": structure,
        "pseudopotentials": pseudopotentials,
        "kpts": (3, 3, 3),
        "calculation": "vc-relax",
        "smearing": 0.02,
    },
    pyiron_project=pr,
)

In [9]:
number_of_strains = 5
structure_lst = generate_structures(  # the generate_structures() function is not available in the workflow graph
    structure=calc_mini.output.structure,
    strain_lst=np.linspace(0.9, 1.1, number_of_strains),
    pyiron_project=pr,
    list_length=number_of_strains,
)

In [10]:
job_strain_lst = []
for i, structure_strain in enumerate(structure_lst):
    calc_strain = calculate_qe(
        working_directory="strain_" + str(i),
        input_dict={
            "structure": structure_strain,
            "pseudopotentials": pseudopotentials,
            "kpts": (3, 3, 3),
            "calculation": "scf",
            "smearing": 0.02,
        },
        pyiron_project=pr,
    )
    job_strain_lst.append(calc_strain)

In [11]:
plot = plot_energy_volume_curve(
    volume_lst=[job.output.volume for job in job_strain_lst],
    energy_lst=[job.output.energy for job in job_strain_lst],
    pyiron_project=pr,
)

In [12]:
write_workflow_json(delayed_object=plot, file_name=workflow_json_filename)

In [13]:
!cat {workflow_json_filename}

{"nodes": {"0": "quantum_espresso_workflow.plot_energy_volume_curve", "1": "python_workflow_definition.shared.get_list", "2": "quantum_espresso_workflow.calculate_qe", "3": "python_workflow_definition.shared.get_dict", "4": "quantum_espresso_workflow.generate_structures", "5": "quantum_espresso_workflow.calculate_qe", "6": "python_workflow_definition.shared.get_dict", "7": "quantum_espresso_workflow.get_bulk_structure", "8": "quantum_espresso_workflow.calculate_qe", "9": "python_workflow_definition.shared.get_dict", "10": "quantum_espresso_workflow.calculate_qe", "11": "python_workflow_definition.shared.get_dict", "12": "quantum_espresso_workflow.calculate_qe", "13": "python_workflow_definition.shared.get_dict", "14": "quantum_espresso_workflow.calculate_qe", "15": "python_workflow_definition.shared.get_dict", "16": "python_workflow_definition.shared.get_list", "17": "strain_0", "18": "mini", "19": "Al", "20": 4.05, "21": true, "22": {"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}, "23": [3, 3,

## Load Workflow with aiida

In [14]:
from aiida import load_profile

load_profile()

Profile<uuid='0911f03384b24f139c7cf96149242165' name='adis'>

In [15]:
from python_workflow_definition.aiida import load_workflow_json

In [16]:
wg = load_workflow_json(file_name=workflow_json_filename)

In [ ]:
result = wg.run()

03/22/2025 08:34:06 AM <193> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [16|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/22/2025 08:34:07 AM <193> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [16|WorkGraphEngine|continue_workgraph]: tasks ready to run: pickle_node18,pickle_node19,pickle_node20,pickle_node21,pickle_node22,pickle_node23,pickle_node24,pickle_node25,pickle_node26,pickle_node27,pickle_node28,pickle_node29,pickle_node30,pickle_node31,pickle_node32
03/22/2025 08:34:07 AM <193> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [16|WorkGraphEngine|run_tasks]: Run task: pickle_node18, type: PYTHONJOB


------------------------------------------------------------
kwargs:  {'value': 'strain_0'}


03/22/2025 08:34:08 AM <193> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [16|WorkGraphEngine|run_tasks]: Run task: pickle_node19, type: PYTHONJOB


------------------------------------------------------------
kwargs:  {'value': 'mini'}


03/22/2025 08:34:09 AM <193> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [16|WorkGraphEngine|run_tasks]: Run task: pickle_node20, type: PYTHONJOB


...

03/22/2025 08:34:18 AM <193> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [16|WorkGraphEngine|run_tasks]: Run task: pickle_node31, type: PYTHONJOB


------------------------------------------------------------
kwargs:  {'value': 'strain_3'}


03/22/2025 08:34:20 AM <193> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [16|WorkGraphEngine|run_tasks]: Run task: pickle_node32, type: PYTHONJOB


------------------------------------------------------------
kwargs:  {'value': 'strain_4'}


03/22/2025 08:34:21 AM <193> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [16|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 25, 33, 41, 49, 57, 65, 73, 81, 89, 97, 105, 113, 121, 129, 137
03/22/2025 08:34:37 AM <193> aiida.scheduler.direct: [WARNING] Unrecognized job_state 'p' for job id Activating
03/22/2025 08:34:37 AM <193> aiida.engine.transports: [ERROR] Exception whilst using transport:
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/aiida/engine/transports.py", line 106, in request_transport
    yield transport_request.future
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/aiida/engine/processes/calcjobs/manager.py", line 109, in _get_jobs_from_scheduler
    scheduler_response = scheduler.get_jobs(**kwargs)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/aiida/schedulers/plugins/direct.py", line 269, in 

In [ ]:
from IPython.display import Image, display

plot_task = [t for t in wg.tasks if t.name.startswith('plot_energy_volume_curve')][0]
plot_file = f"{plot_task.node.get_remote_workdir()}/evcurve.png"

display(Image(filename=str(plot_file)))


## Load Workflow with jobflow

In [ ]:
from python_workflow_definition.jobflow import load_workflow_json

In [ ]:
from jobflow.managers.local import run_locally

In [ ]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [ ]:
result = run_locally(flow)
result